In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import erf
from tqdm import tqdm

In [3]:
def reading_csv(file_path):
    df = pd.read_csv(file_path)
    mean_used = df['S2_val'].tolist()
    result    = df['Decision (S1>S2)'].tolist()
    data_tuple = (result, mean_used)
    return data_tuple

def data_var2(file_path):
    df = pd.read_csv(file_path)
    var2 = df['S2_std'].to_numpy()
    var2 = pd.unique(var2)     # valeurs uniques de S2_std
    return list(var2)

def psycometrique(data_tuple, var2, block_size=1500, big_blocs=11):
    """
    Reconstruit les moyennes par bloc comme dans ton code :
    - true_array_result[j][k] = P(1) pour le j-ième var2, k-ième S2_val
    - true_array_mean_used[j][k] = S2_val moyenne correspondante
    """
    result, mean_used = data_tuple
    true_array_mean_used = []
    true_array_result    = []

    size_one_block = block_size * big_blocs  # nb essais par courbe (11 x 200)

    for j in range(len(var2)):
        array_mean_result = []
        array_mean_used   = []
        for i in range(0, size_one_block, block_size):
            # bloc de result
            block_r = result[size_one_block*j + i : size_one_block*j + i + block_size]
            mean_r  = sum(block_r) / len(block_r)
            array_mean_result.append(mean_r)

            # bloc de mean_used
            block_m = mean_used[size_one_block*j + i : size_one_block*j + i + block_size]
            mean_m  = sum(block_m) / len(block_m)
            array_mean_used.append(mean_m)

        true_array_result.append(array_mean_result)
        true_array_mean_used.append(array_mean_used)

    return true_array_result, true_array_mean_used

def build_targets_from_csv(file_path, block_size=1500, big_blocs=11):
    """
    Construit :
      - target[(mu2, std2)] = P_empirique(réponse=1)
      - mus_test : liste ordonnée des S2_val utilisés
      - vars_test : liste ordonnée des S2_std utilisés
    à partir de ton CSV.
    """
    data_tuple = reading_csv(file_path)
    var2_list  = data_var2(file_path)           # S2_std uniques
    array_mean_result, array_mean_used = psycometrique(
        data_tuple, var2_list,
        block_size=block_size,
        big_blocs=big_blocs
    )

    target = {}
    # on suppose que les S2_val sont les mêmes pour chaque var2
    mus_test = array_mean_used[0]
    vars_test = var2_list

    for j, std2 in enumerate(var2_list):
        for k, mu2 in enumerate(array_mean_used[j]):
            p_emp = array_mean_result[j][k]
            target[(mu2, std2)] = p_emp

    return target, mus_test, vars_test

In [4]:
target, mus_test, vars_test = build_targets_from_csv('C:\\Users\\gabri\\Desktop\\bayesian\\experiment_results_1500.csv') # before experiment_results_test.csv 
print(target)

{(-10.0, np.int64(0)): 1.0, (-8.0, np.int64(0)): 1.0, (-6.0, np.int64(0)): 1.0, (-4.0, np.int64(0)): 1.0, (-2.0, np.int64(0)): 0.9993333333333333, (0.0, np.int64(0)): 0.4746666666666667, (2.0, np.int64(0)): 0.005333333333333333, (4.0, np.int64(0)): 0.0, (6.0, np.int64(0)): 0.0, (8.0, np.int64(0)): 0.0, (10.0, np.int64(0)): 0.0, (-10.0, np.int64(2)): 1.0, (-8.0, np.int64(2)): 0.9986666666666667, (-6.0, np.int64(2)): 0.9953333333333333, (-4.0, np.int64(2)): 0.9566666666666667, (-2.0, np.int64(2)): 0.8313333333333334, (0.0, np.int64(2)): 0.5873333333333334, (2.0, np.int64(2)): 0.346, (4.0, np.int64(2)): 0.11333333333333333, (6.0, np.int64(2)): 0.04133333333333333, (8.0, np.int64(2)): 0.008, (10.0, np.int64(2)): 0.0, (-10.0, np.int64(4)): 0.9526666666666667, (-8.0, np.int64(4)): 0.9266666666666666, (-6.0, np.int64(4)): 0.8886666666666667, (-4.0, np.int64(4)): 0.8313333333333334, (-2.0, np.int64(4)): 0.73, (0.0, np.int64(4)): 0.642, (2.0, np.int64(4)): 0.536, (4.0, np.int64(4)): 0.454, (6.0

In [ ]:
mus_test  = [-10,-8,-6,-4,-2,0,2,4,6,8,10]
vars_test = [0,2,4,6,8]
mu1,mu2 = 0 , 2
var1,var2 = 0.2 , 4
print(target[(mu2, var2)])  # exemple d'accès à une valeur cible

0.536


In [ ]:
from scipy.special import erfinv

def inverse_phi(x):
    """ Approximation de l'inverse de la fonction de répartition normale centrée réduite """
    return np.sqrt(2) * erfinv(2 * x - 1)

